In [9]:
from bs4 import BeautifulSoup, Comment
import os

import spacy
from spacy.lang.en.examples import sentences 
nlp = spacy.load('en_core_web_sm')

def parse(sentence):
    doc = nlp(sentence)
    return [(X.text, X.label_) for X in doc.ents]

In [10]:
def split_records(stream):
    payload = ''
    for line in stream:
        if line.strip() == 'WARC/1.0':
            yield payload
            payload = ''
        else:
            payload += line

In [11]:
def find_key(payload):
    key = None
    for line in payload.splitlines():
        if line.startswith("WARC-TREC-ID"):
            key = line.split(': ')[1]
            return key
    return ''

In [12]:
def record2html(record):
### find html in warc file
    ishtml = False
    html = ""
    for line in record.splitlines():
        ### html starts with <html
        if line.startswith("<html"):
            ishtml = True
        if ishtml:
            html += line
    return html

In [13]:
def html2text(record):
    html_doc = record2html(record)
    # Rule = "/<.*>/";
    useless_tags = ['footer', 'header', 'sidebar', 'sidebar-right', 'sidebar-left', 'sidebar-wrapper', 'wrapwidget', 'widget']
    if html_doc:
        soup = BeautifulSoup(html_doc,"html.parser");
        ### remove tags: <script> <style> <code> <title> <head>
        [s.extract() for s in soup(['script','style', 'code','title','head','footer','header'])]
        ### remove tags id= useless_tags
        [s.extract() for s in soup.find_all(id = useless_tags)]
        ### remove tags class = useless_tags
        [s.extract() for s in soup.find_all(name='div',attrs={"class": useless_tags})]
        ### remove comments
        for element in soup(s=lambda s: isinstance(s, Comment)):
            element.extract()
        # text = soup.get_text("\n", strip=True)

        ### get text in <p></p>
        paragraph = soup.find_all("p")
        text = ""
        for p in paragraph:
            if p.get_text(" ", strip=True) != '':
                text += p.get_text(" ", strip=True)+"\n"
        if text ==  "":
            text = soup.get_text(" ", strip=True)
        # text = re.sub(Rule, "", text)
        # escape character
        # soup_sec = BeautifulSoup(text,"html.parser")

        return text
    return ""

In [26]:
with open('0000tw-00.warc.565132',errors = 'ignore') as f:
    for record in split_records(f):
        key = find_key(record)
        if key != '':
            #print(key)
            html = html2text(record)
            #print(html)
            doc = nlp(html.replace('\"',' ').replace('\'',' ').replace('|'',' ').replace('',' '))
            print([(X.text, X.label_) for X in doc.ents])

[('shaggyshoo', 'ORG'), ('annecy', 'PERSON'), ('france', 'GPE'), ('2008', 'DATE'), ('ds el est', 'PERSON'), ('one', 'CARDINAL'), ('WP Cumulus Flash', 'PRODUCT'), ('Roy Tanck', 'PERSON'), ('Luke Morton', 'PERSON'), ('9', 'CARDINAL')]
[('bleumarie', 'PERSON'), ('2008', 'DATE'), ('ds el est', 'PERSON'), ('one', 'CARDINAL'), ('WP Cumulus Flash', 'PRODUCT'), ('Roy Tanck', 'PERSON'), ('Luke Morton', 'PERSON'), ('9', 'CARDINAL')]
[('bleumarie', 'PERSON'), ('2008', 'DATE'), ('ds el est', 'PERSON'), ('one', 'CARDINAL'), ('WP Cumulus Flash', 'PRODUCT'), ('Roy Tanck', 'PERSON'), ('Luke Morton', 'PERSON'), ('9', 'CARDINAL')]
[('bleumarie', 'PERSON'), ('2008', 'DATE'), ('ds el est', 'PERSON'), ('one', 'CARDINAL'), ('WP Cumulus Flash', 'PRODUCT'), ('Roy Tanck', 'PERSON'), ('Luke Morton', 'PERSON'), ('9', 'CARDINAL')]
[('bleumarie', 'PERSON'), ('2008', 'DATE'), ('ds el est', 'PERSON'), ('one', 'CARDINAL'), ('WP Cumulus Flash', 'PRODUCT'), ('Roy Tanck', 'PERSON'), ('Luke Morton', 'PERSON'), ('9', 'CAR

[('Berlin', 'GPE'), ('Berlin Patient', 'ORG'), ('five', 'CARDINAL'), ('six', 'CARDINAL'), ('millions', 'CARDINAL'), ('first', 'ORDINAL'), ('February 1, 2012', 'DATE'), ('9:02', 'QUANTITY'), ('Blood Cancer', 'ORG'), ('Bone Cancer', 'PERSON'), ('Brain Cancer', 'PERSON'), ('Breast Cancer', 'ORG'), ('Cancer Information', 'ORG'), ('Colorectal Cancer', 'PERSON'), ('Esophageal Cancer', 'PERSON'), ('Health Care Health Insurance', 'ORG'), ('Health Care Services', 'ORG'), ('Health Insurance Benefits', 'ORG'), ('Health Insurance Coverage , Health Insurance Medical , Health Insurance Medical Insurance , Health Insurance Plan , Health', 'ORG'), ('Liver Cancer', 'PERSON'), ('Medical Health Insurance', 'ORG'), ('Mouth', 'PERSON'), ('Ovarian', 'NORP'), ('Skin Cancer', 'PERSON'), ('Stomach Cancer', 'PERSON'), ('Symptoms Of Cancer', 'ORG'), ('Testicular', 'ORG'), ('Cancer', 'ORG'), ('United', 'ORG'), ('Breast Cancer', 'ORG'), ('Women For Women International', 'ORG'), ('Women Health Clinic', 'ORG'), ('He

[('Health Care Technology', 'ORG'), ('The American Association of Retired Persons', 'ORG'), ('September 2008', 'DATE'), ('two', 'CARDINAL'), ('Baby Boomers', 'ORG'), ('43', 'DATE'), ('61', 'DATE'), ('age 65', 'DATE'), ('House', 'ORG'), ('first', 'ORDINAL'), ('February 4, 2012', 'DATE'), ('1:47 pm', 'TIME'), ('Health Care Technology Tags', 'ORG'), ('Blood Cancer', 'ORG'), ('Bone Cancer', 'PERSON'), ('Brain Cancer', 'PERSON'), ('Breast Cancer', 'ORG'), ('Cancer Information', 'ORG'), ('Colorectal Cancer', 'PERSON'), ('Esophageal Cancer', 'PERSON'), ('Health Care Health Insurance', 'ORG'), ('Health Care Services', 'ORG'), ('Health Insurance Benefits', 'ORG'), ('Health Insurance Coverage , Health Insurance Medical , Health Insurance Medical Insurance , Health Insurance Plan , Health', 'ORG'), ('Liver Cancer', 'PERSON'), ('Medical Health Insurance', 'ORG'), ('Mouth', 'PERSON'), ('Ovarian', 'NORP'), ('Private Health Care', 'ORG'), ('Stomach Cancer', 'PERSON'), ('Symptoms Of Cancer', 'ORG'), (

[('every year', 'DATE'), ('One', 'CARDINAL'), ('One', 'CARDINAL'), ('Leukemia', 'PERSON'), ('Leukemia', 'PRODUCT'), ('Leukemia', 'PERSON'), ('Lymphoma', 'GPE'), ('one', 'CARDINAL'), ('Hodgkin', 'GPE'), ('Hodgkin', 'GPE'), ('non-Hodgkin', 'NORP'), ('third', 'ORDINAL'), ('Myeloma', 'ORG'), ('first', 'ORDINAL'), ('February 8, 2012', 'DATE'), ('7:06 pm', 'TIME'), ('Blood Cancer', 'ORG'), ('Bone Cancer', 'PERSON'), ('Brain Cancer', 'PERSON'), ('Breast Cancer', 'ORG'), ('Cancer Information', 'ORG'), ('Colorectal Cancer', 'PERSON'), ('Esophageal Cancer', 'PERSON'), ('Health Care Health Insurance', 'ORG'), ('Health Care Services', 'ORG'), ('Health Insurance Benefits', 'ORG'), ('Health Insurance Coverage , Health Insurance Medical , Health Insurance Medical Insurance , Health Insurance Plan , Health', 'ORG'), ('Liver Cancer', 'PERSON'), ('Medical Health Insurance', 'ORG'), ('Mouth', 'PERSON'), ('Ovarian', 'NORP'), ('Private Health Care', 'ORG'), ('Stomach Cancer', 'PERSON'), ('Symptoms Of Cance

[('Unik Mercedes-Benz', 'PERSON'), ('Klip MDNA Madonna', 'PERSON'), ('Geve Me Al', 'PERSON'), ('Islam Mengg', 'ORG'), ('05.02.2012 Pilot Lion Air Dibekuk', 'ORG'), ('Video Detail', 'PRODUCT'), ('Secepatnya Diri', 'PERSON'), ('Ditutup Se', 'PERSON'), ('Apa Itu SOPA', 'PERSON'), ('App Facebook Pesa', 'PERSON'), ('14.01.2012 Street Fighter IV', 'ORG'), ('12.01.2012 Ikuti Segera', 'PRODUCT'), ('Facebook', 'ORG'), ('2012', 'DATE'), ('Hacke', 'PERSON'), ('60%', 'PERCENT'), ('BlackBerry PlayBook', 'ORG'), ('The Bourne Legacy Baru Tidak', 'PRODUCT'), ('Lagi Di', 'PERSON'), ('2', 'CARDINAL'), ('Anti Virus', 'PERSON'), ('31', 'DATE'), ('24', 'CARDINAL'), ('Browser', 'PERSON'), ('14', 'CARDINAL'), ('Debian', 'NORP'), ('1', 'CARDINAL'), ('29', 'CARDINAL'), ('18', 'CARDINAL'), ('Fedora', 'PERSON'), ('Fennec', 'ORG'), ('1', 'CARDINAL'), ('3', 'CARDINAL'), ('51', 'CARDINAL'), ('1', 'CARDINAL'), ('5', 'CARDINAL'), ('1', 'CARDINAL'), ('56', 'CARDINAL'), ('4', 'CARDINAL'), ('GIMP', 'PERSON'), ('1', 'CAR

...

[('March 2012', 'DATE'), ('Delaware', 'GPE'), ('Valley Americans', 'NORP'), ('United', 'ORG'), ('Philadelphia Committee', 'ORG'), ('Thursday March 29th', 'DATE'), ('7pm', 'TIME'), ('the Skyline Room', 'LOC'), ('4th', 'ORDINAL'), ('the Parkway Central Library', 'FAC'), ('Philadelphia', 'GPE'), ('1901', 'DATE'), ('Vine Street', 'FAC'), ('between 19th and 20th', 'DATE'), ('THE GOOD NEWS CLUB:', 'WORK_OF_ART'), ('Christian', 'NORP'), ('Pennsylvania', 'GPE'), ('House', 'ORG'), ('the “Year of the Bible”', 'EVENT'), ('Lawrence Curry', 'PERSON'), ('Vincent Hughes', 'PERSON'), ('Congresswoman Allyson Schwartz', 'PERSON'), ('Earlier this week', 'DATE'), ('the House of Representatives', 'ORG'), ('535', 'CARDINAL'), ('2012', 'CARDINAL'), ('Rick Saccone', 'PERSON'), ('Râ€“Elizabeth Township', 'GPE'), ('36', 'CARDINAL'), ('the Constitution of the\nGreetings', 'LAW'), ('Iâ€™m', 'MONEY'), ('first', 'ORDINAL'), ('Clergy Advisory', 'ORG'), ('Tuesday, February 21', 'DATE'), ('6:00', 'TIME'), ('the Ethica

[('Pennsylvania Legislators', 'ORG'), ('House', 'ORG'), ('the “Year of the Bible”', 'EVENT'), ('Lawrence Curry', 'PERSON'), ('Vincent Hughes', 'PERSON'), ('Congresswoman Allyson Schwartz', 'PERSON'), ('Pennsylvania General Assembly', 'ORG'), ('House Resolution No', 'ORG'), ('535', 'CARDINAL'), ('January 24, 2012', 'DATE'), ('House Resolution No', 'ORG'), ('535', 'CARDINAL'), ('the year of 2012', 'DATE'), ('Bible', 'WORK_OF_ART'), ('the State of Pennsylvania', 'GPE'), ('the Separation of Church', 'ORG'), ('State', 'ORG'), ('California', 'GPE'), ('Rhode Island', 'GPE'), ('Pennsylvania', 'GPE'), ('the last few months', 'DATE'), ('Pennsylvania', 'GPE'), ('Christian', 'NORP'), ('one', 'CARDINAL'), ('the Pennsylvania State Constitution', 'ORG'), ('the Constitution of United States of America', 'LAW'), ('Pennsylvanians', 'NORP'), ('Christian', 'NORP'), ('Islam', 'GPE'), ('Mormon', 'GPE'), ('Universal Unitarian', 'PERSON'), ('Eastern', 'ORG'), ('The Gen. Assembly', 'ORG'), ('the Declaration of

[('Love Sacks', 'WORK_OF_ART'), ('1', 'CARDINAL'), ('twelve-pound', 'QUANTITY'), ('under 4 feet', 'QUANTITY'), ('one', 'CARDINAL'), ('two to three', 'CARDINAL'), ('5 to eight feet', 'QUANTITY'), ('a few feet', 'QUANTITY'), ('40 to 95 pounds', 'QUANTITY'), ('about 35 pounds', 'QUANTITY'), ('5', 'CARDINAL'), ('one', 'CARDINAL'), ('Love Sacks', 'PERSON'), ('Warren Harmon', 'PERSON')]
[('Glass', 'PERSON'), ('200c', 'PRODUCT'), ('500f', 'CARDINAL'), ('1600f', 'DATE'), ('Glass', 'PERSON'), ('30 minute', 'TIME'), ('FD30', 'ORG'), ('Ian Richardson', 'PERSON'), ('1', 'CARDINAL'), ('UK', 'GPE'), ('1', 'CARDINAL'), ('the last 15 years 1', 'DATE'), ('1', 'CARDINAL'), ('UK', 'GPE'), ('FD30 & FD', 'ORG'), ('60', 'CARDINAL'), ('one', 'CARDINAL'), ('7 days', 'DATE'), ('each  week', 'DATE'), ('8am', 'TIME'), ('7 days', 'DATE'), ('Ian Richardson', 'PERSON'), ('Subscribe', 'PERSON')]
[]
[('a 5 years old', 'DATE'), ('the first 3 years', 'DATE'), ('70%', 'PERCENT'), ('four years old to five', 'DATE'), ('20

[('Grammy week', 'EVENT'), ('third', 'ORDINAL'), ('annual', 'DATE'), ('Black Woman', 'EVENT'), ('last night', 'TIME'), ('two', 'CARDINAL'), ('Syliva Rhone', 'ORG'), ('Kelly Rowland', 'PERSON'), ('the Belasco hotel', 'FAC'), ('LA', 'GPE'), ('Holly Robinson Peete', 'PERSON'), ('Latoya Luckett', 'PERSON'), ('Missy Elliot', 'PERSON'), ('Sylvia Rhone', 'PERSON'), ('Rhone', 'PERSON'), ('first', 'ORDINAL'), ('Elektra Entertainment Group', 'ORG'), ('Rhone', 'ORG'), ('Missy Elliot', 'PERSON'), ('MC Lyte', 'PERSON'), ('Nicki Minaj', 'ORG'), ('Kelly Rowland', 'PERSON'), ('second', 'ORDINAL'), ('Eve', 'PERSON'), ('Like This’', 'WORK_OF_ART'), ('Rico Love', 'PERSON'), ('Kelley', 'PERSON'), ('couldnâ€™', 'DATE'), ('Kelly Rowland', 'PERSON'), ('Shawn Sutton', 'PERSON'), ('Ne-Yo', 'PERSON'), ('one', 'CARDINAL'), ('Michelle Williams', 'PERSON'), ('Kelly', 'PERSON'), ('Letoya Luckett', 'PERSON'), ('Toccara Dawn Richard', 'PERSON'), ('Model Eva Marcille', 'PERSON'), ('Naven', 'ORG'), ('Louboutin', 'PERSO

[('Kanye West', 'GPE'), ('Jay-Z', 'PERSON'), ('summer', 'DATE'), ('Paris', 'GPE'), ('one', 'CARDINAL'), ('the Watch', 'WORK_OF_ART'), ('Jay', 'PERSON'), ('Ye', 'PERSON'), ('more than 8', 'CARDINAL'), ('Jay', 'PERSON'), ('Ye', 'PERSON'), ('Parisian', 'NORP'), ('Epilepsy', 'PERSON'), ('February 9, 2012', 'DATE'), ('5:03 pm', 'TIME'), ('One', 'CARDINAL'), ('February 9th, 2012', 'DATE'), ('5:09 pm', 'TIME'), ('February 9th, 2012', 'DATE'), ('5:17 pm', 'TIME'), ('Jay & Kanye', 'ORG'), ('Reply', 'ORG'), ('Reply', 'ORG'), ('February 9th, 2012', 'DATE'), ('5:45 pm', 'TIME'), ('February 9th, 2012', 'DATE'), ('8:47 pm', 'TIME'), ('luis vuitton', 'PERSON'), ('Reply', 'ORG'), ('February 9th, 2012', 'DATE'), ('9:13 pm', 'TIME'), ('jay', 'PERSON'), ('kanye', 'GPE'), ('February 10th, 2012', 'DATE'), ('1:50 am', 'TIME'), ('StampedSuspect', 'PERSON'), ('February 10th, 2012', 'DATE'), ('1:51 am', 'TIME'), ('February 9th, 2012', 'DATE'), ('9:34 pm', 'TIME'), ('Epilepsy', 'PERSON'), ('Reply', 'ORG'), ('Fe

[('shaggyshoo', 'ORG'),
 ('annecy', 'PERSON'),
 ('one', 'CARDINAL'),
 ('Roy Tanck', 'PERSON'),
 ('Luke Morton', 'PERSON'),
 ('9', 'CARDINAL')]